# Awkward Array quickstart
From the awkward array documentation found [here](https://awkward-array.org/doc/main/), "Awkward Array is a library for nested, variable-sized data, including arbitrary-length lists, records, mixed types, and missing data, using NumPy-like idioms." Awkward array's API makes columnar analysis straightforward to implement in Python. Columnar analysis is advantageous to row-wise analysis as it drastically reduces run time by utilizing memory more effectively and reducing the total number of computer instructions. In this section, we will discuss building awkward arrays and some simple operations. 

## Creating an Awkward Array
First `import awkward` and create a simple jagged array.

In [2]:
import awkward as ak

In [3]:
array = ak.Array([[5,4,4],[2],[3,5]])
array

<Array [[5, 4, 4], [2], [3, 5]] type='3 * var * int64'>

Awkward array entries can be read the same way as a python list or numpy array with indexing. Lets read the 3rd row's 2nd entry as an example.

In [4]:
print(array[2,1])

5


# Matrix Manipulations



## Array Arithmetic
Let's perform some simple manipulations with this array. It is possible to add arrays, multiple arrays element wise, and multiply arrays by a scalar. Check that the below result makes sense with pen and paper.

In [5]:
array*array+5*array

<Array [[50, 36, 36], [14], [24, 50]] type='3 * var * int64'>

The line "type: 3 * var * int64" is read as follows. The array has 3 entries of variable (var) length, where the entries are of datatype int64. Take a look at the two examples below, can you understand the **"type"** output?

In [6]:
array1 = ak.Array([1.,2.,3.])
array1

<Array [1, 2, 3] type='3 * float64'>

In [7]:
array2 = ak.Array([[2,3.2],[[2,3],[5,6], [10,10]]])
array2

<Array [[2, 3.2], [[2, 3], ..., [...]]] type='2 * var * union[float64, var ...'>

## Products and Sums (Reducers)

Suppose we want to multiply the values in a row together. In awkward array this is accomplished by calling `ak.prod()` and defining which axis to multiply along. `axis=1` corresponds to multiplying the elements of each inner array.

In [8]:
row_product = ak.prod(array, axis=1)
print("Row Product: \t", row_product, " Row Product dimension: \t", row_product.ndim)
column_product = ak.prod(array, axis = 0)
print("Column Product: ", column_product, " Column Product dimension: ", column_product.ndim)

Row Product: 	 [80, 2, 15]  Row Product dimension: 	 1
Column Product:  [30, 20, 4]  Column Product dimension:  1



If you instead set `axis=0`, it will multiply the elements along the columns of this array. This operation has reduced the dimension of the array from 2 to 1, hence this operation is a 'reducer'. It is neccesarry to keep in mind that for certain operations between awkward arrays to work, they must be of the same dimension. If instead `ak.prod` returned:


In [9]:
unflattened_row_product = ak.Array([[80],[2],[15]])
unflattened_row_product

<Array [[80], [2], [15]] type='3 * var * int64'>

We may need to reduce the resulting array dimension somehow for the analysis with a function like `ak.flatten()`. You may even need to increase the dimension depending on the computation with a function like `ak.singletons()`. Below we can make `row_product` match `unflattened_row_product`.
                                        


In [10]:
ak.singletons(row_product)

<Array [[80], [2], [15]] type='3 * var * int64'>

Similarly to `ak.prod()`, we can sum along a given axis by calling `ak.sum()`.

In [11]:
row_sum = ak.sum(array, axis=1)
row_sum

<Array [13, 2, 8] type='3 * int64'>

## Boolean Operations
Boolean operations can be performed on awkward arrays element-wise. This is often done when evaluating selection criteria. Let's obtain an array of booleans containing the information of which entries are $> 3$.

In [12]:
selection = array > 3
selection

<Array [[True, True, True], [False], [False, True]] type='3 * var * bool'>

With `selection`, we can obtain information about which rows contain at least one entry satisfying the criterion by calling `ak.any()`.

In [13]:
good_row = ak.any(selection, axis = 1)
good_row

<Array [True, False, True] type='3 * bool'>

`ak.any()` is analogous to an OR operation, and the method `ak.all()` is analogous to an AND operation. 

In [14]:
ak.all(selection, axis = 1)

<Array [True, False, False] type='3 * bool'>

# Masking (Applying Selection Cuts)
Now consider that we may want to remove entries that dont match our criterion. This is done by masking the array. We pass an array of booleans that define what entries to keep or exclude. Masking can be done at any depth of the array. Let's first 'cut' out all the entries less than 3 and compare the original array with the new cut array.

In [15]:
# Recalculate masking array
selection = array > 3
#Pass cut as an array of booleans
masked_array = array[selection]
print("Original array", array)
print("Cut array", masked_array)

Original array [[5, 4, 4], [2], [3, 5]]
Cut array [[5, 4, 4], [], [5]]


The entries with value less than or equal to 3 have been removed from the array. If we also want to remove the rows without any entries, we can do:

In [16]:
rows_pass = ak.any(selection, axis = 1)
masked_array = masked_array[rows_pass]
print(masked_array)

[[5, 4, 4], [5]]


# Building Arrays of Records

"In Awkward Array, a “record” is a structure containing a fixed-length set of typed, possibly named fields." For LHC data, an event is akin to a record. Let's build up a representative awkward array of data you might get from `Uproot` of muon data.

From a list of dictionaries.

In [17]:
content_dict = [
    {"muon_pt" : 101.2, "muon_eta" : 0.1, "muon_phi" : -1.2},
    {"muon_pt" : 56.7, "muon_eta" : 2.3, "muon_phi" : 3.1},
    {"muon_pt" : 3.2, "muon_eta" : -1.4, "muon_phi" : 0.56},
]
muon_data = ak.Array(content_dict)
muon_data

<Array [{muon_pt: 101, ...}, {...}, {...}] type='3 * {muon_pt: float64, muo...'>

We can pick out a "field" by passing the field name in brackets to the array.

In [18]:
muon_data["muon_pt"]

<Array [101, 56.7, 3.2] type='3 * float64'>

The same can be done by requesting the array attribute for a given field

In [19]:
muon_data.muon_pt

<Array [101, 56.7, 3.2] type='3 * float64'>

The dictionary syntax albeit clear is unweildly for large sets of data. Alternatively, we can build arrays with `ak.zip()` from columns of data which is more intuitive when working with data.

In [21]:
muon_pt = [None, 56.7, 3.2]
muon_eta = [0.1, 2.3, -1.4]
muon_phi = [-1.2, 3.1, 0.56]
muon_data_zip = ak.zip(
    {
        "muon_pt": muon_pt,
        "muon_eta": muon_eta,
        "muon_phi": muon_phi,
    }
)
muon_data_zip

<Array [{muon_pt: None, ...}, ..., {...}] type='3 * {muon_pt: ?float64, muo...'>

# Conclusion
Awkward array is just as capable as other data structures at performing calculations and filtering data. The operations shown will cover most of the techniques needed for an analysis. Now we will get some practice working with awkward arrays.

___
# Exercises

## Exercise 1: 
Compute the sum of all odd-valued integers of the array below. Do so using python 'for loops' and then again using awkward array's API to implement a columnar approach. How much of a speed-up does using awkward array give compared to using for loops? Verify that your result make sense and agrees between the two methods.

In [24]:
import numpy as np
import awkward as ak
# Create a sample NumPy array
dtype = np.int32
shape = (100000)
data = np.random.default_rng().integers(low=0,high=10,size = shape)
ak_data = ak.Array(data)
print(ak_data)

[0, 6, 3, 1, 6, 3, 0, 1, 5, 8, 9, 6, 2, ..., 2, 7, 4, 3, 4, 8, 5, 6, 7, 8, 6, 0]


In [25]:
%%time
#row-wise
sum_row = 0
for element in ak_data:
    if (element % 2) == 1:
        sum_row += element

sum_row

CPU times: user 166 ms, sys: 716 μs, total: 167 ms
Wall time: 166 ms


np.int64(250250)

In [26]:
%%time
#column-wise
sum_columnar = 0
mask = (ak_data % 2) == 1
masked_data = ak_data[mask]
sum_columnar = ak.sum(masked_data)
# Fix me

sum_columnar

CPU times: user 4.22 ms, sys: 2.39 ms, total: 6.61 ms
Wall time: 5.04 ms


np.int64(250250)

In [27]:
ak.sum(ak_data)

np.int64(450070)

In [ ]:
# Compare results of two methods
assert (sum_columnar == sum_row) ## Fix-me

## Exercise 2: 
Using weather data from the KCLT, KMDW, KCQT, and KSEA weather stations determine the number of days with above average temperatures, record setting high temperatures, and record setting precipitation for each. Then how does the actual total precipitation compare to the average total precipitation? Do you notice any irregular weather patterns for the time period?

Plot the actual, average, and record min/max temperatures. Plot the actual, average, and record precipitation. Does this change your answers from above?

Note: Your solution should only use awkward array's API.

Inspiration taken from: https://github.com/fivethirtyeight/data/tree/master/us-weather-history

In [29]:
ak_data_MDW.fields

['date',
 'actual_mean_temp',
 'actual_min_temp',
 'actual_max_temp',
 'average_min_temp',
 'average_max_temp',
 'record_min_temp',
 'record_max_temp',
 'record_min_temp_year',
 'record_max_temp_year',
 'actual_precipitation',
 'average_precipitation',
 'record_precipitation']

In [28]:
import awkward as ak
#Read the data
ak_data_MDW = ak.from_parquet("data/KMDW.parquet")
ak_data_CLT = ak.from_parquet("data/KCLT.parquet")
ak_data_CQT = ak.from_parquet("data/KCQT.parquet")
ak_data_SEA = ak.from_parquet("data/KSEA.parquet")

def run_weather_analysis(ak_data, city):
    print("Analyzing {}'s weather data".format(city))
    ###################################################
    ### a) Find # of days with above record max temperature
    # Fix me
    
    # Result
    print("Days setting a new record high: ", ) #Fix me
    
    ###################################################
    ### b) Find # of days with above average max temperature
    # Fix me
    
    # Result
    print("Days above average max: ", ) #Fix me
    
    ###################################################
    ### c) Find # of days with record setting precipitation
    #Fix me
    
    # Result
    print("Days setting a new record precipitation: ", len(ak_data[record_set_precip]))
    
    ###################################################
    ### d) Total precipitation
    # Fix me

    print("Total actual precipitation: ", total_actual_precip, "in")
    print("Total average precipitation: ", total_average_precip, "in")
    print("\n")
    ###################################################
    
run_weather_analysis(ak_data_MDW, "Chicago, IL")
run_weather_analysis(ak_data_CLT, "Charleston, NC")
run_weather_analysis(ak_data_CQT, "Los Angeles, CA")
run_weather_analysis(ak_data_SEA, "Seattle, WA")

Analyzing Chicago, IL's weather data
Days setting a new record high: 
Days above average max: 


NameError: name 'record_set_precip' is not defined

In [ ]:
import matplotlib.pyplot as plt
### Visualization
def make_plot_temp(ak_data, city, save_file):
    print("Generating {}'s weather data plot".format(city))
    date_order = ak_data['date']

    # Initialize figure, define size
    fig, ax1 = plt.subplots(figsize=(15, 8))
    
    
    # Create the bars showing all-time record highs and lows
    plt.bar(date_order,
            ak_data['record_max_temp'] - ak_data['record_min_temp'],
            bottom=ak_data['record_min_temp'],
            edgecolor='none', color='#C3BBA4', width=1)
    
    # Create the bars showing average highs and lows
    plt.bar(date_order, ak_data["average_max_temp"] - ak_data["average_min_temp"],
            bottom=ak_data["average_min_temp"],
            edgecolor='none', color='#9A9180', width=1) 
    
    #Create bars showing day high/low
    plt.bar(date_order,
            ak_data['actual_max_temp'] - ak_data['actual_min_temp'],
            bottom = ak_data['actual_min_temp'],
            edgecolor = 'black', linewidth = 0.5, color = '#5A3B49', width = 1)
    

    ###################################################
    # e) Plot highs
    
    
    record_set_high = # Fix me (Hint: is a mask)
    highs = ak_data[record_set_high]
    
    plt.scatter(highs["date"], highs["actual_max_temp"] + 1,
               alpha = 0.75, s = 15, linewidth=0, color='r')

    #########################################################################
    # Formatting
    plt.ylabel(r'Temperature ($^\circ$F)', fontsize=12)
    plt.title("{}'s weather, July 2014 - June 2015\n\n".format(city), fontsize=20)
    plt.grid(alpha = 0.5, linestyle = '--')
    plt.savefig(save_file)

make_plot_temp(ak_data_SEA, "Seattle, WA", "plots/KSEA.png")
make_plot_temp(ak_data_MDW, "Chicago, IL", "plots/KMDW.png")
make_plot_temp(ak_data_CLT, "Charleston, NC", "plots/KCLT.png")
make_plot_temp(ak_data_CQT, "Los Angeles, CA", "plots/KCQT.png")

In [ ]:
def make_plot_precip(ak_data, city):
    print("Generating {}'s precipitation data plot".format(city))
    date_order = ak_data['date']
    
    # Initialize figure, define size
    fig, ax1 = plt.subplots(figsize=(15, 7))

    # Grab relevant columns
    record_precip = ak_data["record_precipitation"]
    actual_precip = ak_data["actual_precipitation"]
    average_precip = ak_data["average_precipitation"]
    
    # Create the bars showing all-time record high
    plt.bar(date_order, record_precip,
            edgecolor='none', color='#C3BBA4', width=1)
    
    #Create bars showing average
    plt.bar(date_order, average_precip,
            edgecolor = 'black', linewidth = 0.5, color = '#5A3B49', width = 1)
    
    plt.plot(date_order, actual_precip, color = 'r', linewidth = 0.8)

    #Mark Record days
    record_days = record_precip <= actual_precip
    record_precip_data = ak_data[record_days]
    plt.scatter(record_precip_data["date"], record_precip_data["actual_precipitation"] + 0.2, alpha = 0.75)

    #########################################################################
    # Formatting
    plt.ylabel(r'Precipitation (in)', fontsize=12)
    plt.title("{}'s precipitation, July 2014 - June 2015\n\n".format(city), fontsize=20)
    plt.grid(alpha = 0.5, linestyle = '--')
    plt.ylim(0,4)

make_plot_precip(ak_data_CQT, "Los Angeles, CA")

## Bonus Exercise 3: Fractals
"The Mandelbrot set (/ˈmændəlbroʊt, -brɒt/) is a two-dimensional set with a relatively simple definition that exhibits great complexity, especially as it is magnified. It is popular for its aesthetic appeal and fractal structures. The set is defined in the complex plane as the complex numbers  ${\displaystyle c}$ for which the function ${\displaystyle f_{c}(z)=z^{2}+c}$ does not diverge to infinity when iterated starting at ${\displaystyle z=0}$, i.e., for which the sequence ${\displaystyle f_{c}(0)}$, ${\displaystyle f_{c}(f_{c}(0))}$, etc., remains bounded in absolute value." - Wikipedia

Starting with the numpy implementation, solve the Mandelbrot set with only `awkward` api calls but leave `np.ogrid`. Describe the difference in performance. What differences/similarities in the underlying array objects can you attribute to the performance comparison?

**Hint:** You may need to use `ak.fill_none`, `ak.mask`, `ak.zeros_like`, and `ak.ones_like` in your solution. Please use the awkward API reference guide found here for any API questions. https://awkward-array.org/doc/main/reference/index.html

In [ ]:
%%time
import matplotlib.pyplot as plt
import numpy as np
### Numpy
def run_numpy(height, width, real_min=-2, real_max=0.5, imag_min=-1.2, imag_max=1.2):
    y, x = np.ogrid[imag_min:imag_max:height*1j, real_min:real_max:width*1j]
    c = x + y*1j

    z = np.zeros(c.shape, dtype=np.complex128)       # initial values of all elements of z are 0
    picture = np.full(c.shape, 20, dtype=np.int64)   # initial values of all pixels are 20

    for i in range(20):
        z = z**2 + c
        picture[(picture == 20) & (abs(z) > 2)] = i
        
    return picture

fig, ax = plt.subplots(1, 1, figsize=(15, 10))

with np.errstate(over="ignore", invalid="ignore"):   # ignore warnings about inf and nan values in z
    im = ax.imshow(run_numpy(800, 1600));
    plt.colorbar(im)

In [ ]:
%%time
import matplotlib.pyplot as plt
import awkward as ak
import numpy as np
### Awkward
def run_ak(height, width, real_min=-2, real_max=0.5, imag_min=-1.2, imag_max=1.2):
    y, x = np.ogrid[imag_min:imag_max:height*1j, real_min:real_max:width*1j]
    c = x + y*1j
    z = #Fix me (Hint: initial values of all elements of z are 0)
 
    picture = #Fix me
    picture_out = #Fix me
    
    for i in range(20):
        z = z**2 + c
        picture_out = #Fix me
    
    return picture_out

fig, ax = plt.subplots(1, 1, figsize=(15, 10))

with np.errstate(over="ignore", invalid="ignore"):   # ignore warnings about inf and nan values in z
    im = ax.imshow(run_ak(800, 1600));
    plt.colorbar(im)
    fig.show()